# FreeMASTER Lite (Motor Variable Loader)

This notebook will walk you through the main steps required to establish a connection to FreeMASTER Lite and start communication with the board and load variables.

**Before proceeding to code execution make sure that your setup meets the following requirements:**

### Target

* Embedded application includes FreeMASTER driver
* Specified interface is configured for FreeMASTER communication

### Host

* FreeMASTER Lite is running (the port number should be displayed in console window)

Considering the target board is connected to the host PC you are ready to go.

## Get target info

FreeMASTER Lite exposes a JSON-RPC API via websocket protocol. You need to import the corresponding dependencies in order to run the code.

In [1]:
from datetime import datetime
import usb.core
import usb.util
import usb.backend.libusb1
import sys, os
import asyncio

import socket as s
import time
import subprocess
import datetime
import serial
import serial.tools.list_ports

import websockets
from jsonrpcclient.clients.websockets_client import WebSocketsClient

from pathlib import Path

Adjust the *machine url* if you are connecting to a remote host, and/or *service_port* if it is running on a different port.

In [2]:
machine_url = '127.0.0.1'
service_protocol = 'ws://'
service_port = '8090'
service_url = service_protocol + machine_url + ':' + service_port

Service requires a connection string to establish a communication channel to the target.

In [3]:
#connection = 'RS232;port=COM9;speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'
#connection = 'RS232;port=COM3;speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'
#connection = 'RS232;port=COM6;speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'

def getConnection():
    #Note on USB IDs
    #Vendor ID:	  1357	P&E Microcomputer Systems
    #Device ID:   0089	OpenSDA - CDC Serial Port
    #Looking for "DEVICE ID 1357:0089" using find_class
    venId = 0x1357
    devId = 0x0089
    os.environ['PYUSB_DEBUG'] = 'debug'
    usb.core.find()
    #print([comport.device for comport in serial.tools.list_ports.comports()])
    ports = [comport.device for comport in serial.tools.list_ports.comports()]
    print("ports = ", ports)
    # USB serial port '/dev/ttyACM0'
    if ports != False:
        if str(ports[0]) != 'COM1' and str(ports[0]) != 'COM2':
            cp = str(ports[0])
            connection = 'RS232;port='+ cp +';speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'
        elif str(ports[1]) != 'COM1' and str(ports[1]) != 'COM2':
            cp = str(ports[1])
            connection = 'RS232;port='+ cp +';speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'
        else:
            cp = str(ports[2])
            connection = 'RS232;port='+ cp +';speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'
    else:
        print("Motorboard is not connected")
        connection =''
    return connection

connection = getConnection()
#connection = 'RS232;port=COM3;speed=115200;tmoRI=40;tmoRTM=40;tmoRTC=50;tmoWTM=40;tmoWTC=50;'

ports =  ['COM1', 'COM2', 'COM4']


The function below is a helper method that will encode the command in JSON-RPC format, and send it via websocket client. Once the response is received, it will check the *success* property of the result. If its value is *true* it will return the data content, otherwise - raise an exception. 

In [4]:
async def SendRequest(jrpc, method, *args):
    response = await jrpc.request(method, *args)
    if response.data.result['success']:
        return response.data.result['data']
    else:
        raise Exception(response.data.result['error'])

This block connects you to the server.

In [5]:
ws = await websockets.connect(service_url)
jrpc = WebSocketsClient(ws)

Once you are connected, open a communication port to the target.

In [6]:
await SendRequest(jrpc, 'StartComm', connection)
print('Successfully connected to the target')

Successfully connected to the target


Check if target is detected.

In [7]:
await SendRequest(jrpc, 'IsBoardDetected')
print('Board detected')

Board detected


Query application version.

In [8]:
data = await SendRequest(jrpc, 'GetDetectedBoardInfo') # doesn't work
print('Detected board info: ', data)

Detected board info:  {'protVer': 3, 'cfgFlags': 0, 'dataBusWdt': 1, 'globVerMajor': 2, 'globVerMinor': 0, 'cmdBuffSize': 60, 'recBuffSize': 2048, 'recTimeBase': 32868, 'descr': [83, 51, 50, 120, 120, 32, 70, 114, 101, 101, 77, 65, 83, 84, 69, 82, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


Query application name.

In [9]:
data = await SendRequest(jrpc, 'GetAppVersion') #works
print('GetAppVersion: ',data)

GetAppVersion:  FreeMASTER Lite 1.1 RTM Update 3


## Create Varibles and Write to Motorboard

Read and parse TSA table.

In [10]:
data = await SendRequest(jrpc, 'ReadTSA')
print('Parsed TSA contained ' + str(data['count']) + ' symbols.')
global count
count = int(data['count'])

Parsed TSA contained 342 symbols.


Display found symbols.

In [11]:
for i in range(count):
    data = await SendRequest(jrpc, 'EnumSymbols', i)
    print(data)

CL_SpeedRampDec
CL_SpeedRampInc
FW_IntegGainControl
FW_PropGainControl
OL_SpeedRampInc
UDQVectorSum
cntrState
cntrState.event
cntrState.loadDefSetting
cntrState.state
cntrState.usrControl
cntrState.usrControl.FOCcontrolMode
cntrState.usrControl.btFlipFlop
cntrState.usrControl.btFlipFlopTemp
cntrState.usrControl.btSpeedDown
cntrState.usrControl.btSpeedUp
cntrState.usrControl.cntAppOff
cntrState.usrControl.cntSpeedDown
cntrState.usrControl.cntSpeedUp
cntrState.usrControl.controlMode
cntrState.usrControl.ledCounter
cntrState.usrControl.ledFlashing
cntrState.usrControl.readFault
cntrState.usrControl.switchAppOnOff
cntrState.usrControl.switchAppOnOffState
cntrState.usrControl.switchAppReset
cntrState.usrControl.switchFaultClear
drvFOC
drvFOC.AlBeReqDCBLim
drvFOC.CurrentLoop
drvFOC.CurrentLoop.pIDQFbck
drvFOC.CurrentLoop.pIDQReq
drvFOC.CurrentLoop.pPIrAWD
drvFOC.CurrentLoop.pPIrAWD.fltAcc
drvFOC.CurrentLoop.pPIrAWD.fltCC1sc
drvFOC.CurrentLoop.pPIrAWD.fltCC2sc
drvFOC.CurrentLoop.pPIrAWD.fltIn

pdbStatus.PDB1_SeqErrCounter
pdbStatus.PDB1_SeqErrFlags
permFaults
permFaults.gd3000
permFaults.mcu
permFaults.mcu.R
permFaults.motor
permFaults.motor.R
permFaults.stateMachine
permFaults.stateMachine.R
pos_mode
switchSensor
tempfaults
tempfaults.gd3000
tempfaults.mcu
tempfaults.mcu.R
tempfaults.motor
tempfaults.motor.R
tempfaults.stateMachine
tempfaults.stateMachine.R
tppDrvConfig
tppDrvConfig.csPinIndex
tppDrvConfig.csPinInstance
tppDrvConfig.deviceConfig
tppDrvConfig.deviceConfig.deadtime
tppDrvConfig.deviceConfig.intMask0
tppDrvConfig.deviceConfig.intMask1
tppDrvConfig.deviceConfig.modeMask
tppDrvConfig.deviceConfig.opMode
tppDrvConfig.deviceConfig.statusRegister
tppDrvConfig.deviceConfig.statusRegister[0]
tppDrvConfig.en1PinIndex
tppDrvConfig.en1PinInstance
tppDrvConfig.en2PinIndex
tppDrvConfig.en2PinInstance
tppDrvConfig.rstPinIndex
tppDrvConfig.rstPinInstance
tppDrvConfig.spiInstance
tppDrvConfig.spiTppConfig


Next define variables based one of the symbols.

In [12]:
global variable
speedRequired = {
    "name": "Speed Required", 
    "addr": "drvFOC.pospeControl.wRotElReq", 
    "size": 4, 
    "type": "float"
}
await SendRequest(jrpc, 'DefineVariable', speedRequired)
print('Variable defined successfully.')

Variable defined successfully.


In [13]:
global variable
appOnOff = {
      "name": "On/Off",
      "addr": "cntrState.usrControl.switchAppOnOff",
      "size": 1,
      "type": "uint"
    }
await SendRequest(jrpc, 'DefineVariable', appOnOff)
print('Variable defined successfully.')

Variable defined successfully.


In [14]:
global variable
Mode = {
      "name": "Mode",
      "addr": "cntrState.usrControl.controlMode",
      "size": 1,
      "type": "uint"
    }
await SendRequest(jrpc, 'DefineVariable', Mode)
print('Variable defined successfully.')

Variable defined successfully.


In [15]:
global variable
clearFaults =  {
      "name": "Clear Faults",
      "addr": "cntrState.usrControl.switchFaultClear",
      "size": 1,
      "type": "uint"
    }
await SendRequest(jrpc, 'DefineVariable', clearFaults)
print('Variable defined successfully.')

Variable defined successfully.


In [16]:
response = await SendRequest(jrpc, 'StopComm')
print('Communication stopped')

Communication stopped


End of Code